# Extreme temperature identification

This notebook contains working notes and code to identify T extremes.

My goal is to develop methods that will identify extremes, and then be able to compute various facets of extreme events, such as duration and magnitude.

## Quantile computation

The Xarray quantile method currently does not support dask arrays, so parallelization is not automatic. That means that the calculation of quantiles has to be done on an in-memory DataArray/NumpyArray. I suspect that the implementation of quantile uses numpy's nanpercentile for the calculation. This is known to be slow because it does a 1d calculation looped over all points [https://krstn.eu/np.nanpercentile()-there-has-to-be-a-faster-way/]. 

Based on experimentation in this notebook, I find that `np.nanquantile` is fast enough. 

There is a **major** performance bottleneck in using xarray, however. When I calculate quantiles with DataArray objects, it is very slow; to calculate the 90th percentile for the observations over the 1990s takes more than an hour on topaz.

That relies on applying a function on a groupby object, which is a complication for the real calculation.

If we convert directly to numpy arrays, the basic calculation time drops dramatically. I can calculate a set of quantiles for the same data in 23s. But this did not groupby day-of-year.

In [1]:
%matplotlib inline

In [2]:
import numpy as np 
import xarray as xr

In [3]:
from distributed import Client

In [4]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:39912 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 9 Cores: 72 Memory: 405.33 GB


In [5]:
%%time
ds = xr.open_mfdataset("/project/amp/jcaron/CPC_Tminmax/tmax.199*.nc", decode_cf=False)
ds = xr.decode_cf(ds)
tmax = ds['tmax']
print("Loaded the tmax DataArray (likely lazy load)")

Loaded the tmax DataArray (likely lazy load)
CPU times: user 296 ms, sys: 53 ms, total: 349 ms
Wall time: 1.09 s


In [6]:
%%time
tmax_np = tmax.values
print("Convert to numpy array.")

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 14.3 µs
Convert to numpy array.


In [8]:
%%time
time_ndx = tmax.dims.index('time')

quants = np.nanquantile(tmax_np, [.01, .05, .1, .25, .5, .75, .9, .95, .99], axis=time_ndx, overwrite_input=False, interpolation='linear', keepdims=False)

print("Quantiles Calculated.")



/project/amp/brianpm/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1354: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


Quantiles Calculated.
CPU times: user 24.6 s, sys: 961 ms, total: 25.5 s
Wall time: 23.9 s


In [21]:
%%time
lat = ds['lat']
lon = ds['lon']
time = ds['time']

CPU times: user 649 µs, sys: 0 ns, total: 649 µs
Wall time: 1.06 ms


In [12]:
quants.shape
quantile = [.01, .05, .1, .25, .5, .75, .9, .95, .99]
quants_xr = xr.DataArray(quants, coords={"quantile":quantile, "lat":lat, "lon":lon}, dims=("quantile","lat","lon"))

In [16]:
quants_xr.isel(quantile=-1).max()

<xarray.DataArray ()>
array(50.88431)
Coordinates:
    quantile  float64 0.99

In [20]:
# %%time
# # now we need to try to groupby day of year and then reconstruct.
# grp = tmax.groupby('time.dayofyear')
# grp_dict = dict(grp)
# grp_dict_np = {day: grp_dict[day].values for day in grp_dict}
# print(grp_dict_np)

# TOO SLOW!

In [26]:
%%time
# alternative to using groupby is to get the indices for each day of year
# Makes a dictionary with keys that are day of year and values that are the data
# GREAT START ... skip down to bottom to then use this approach for better sampling

doy = set(time.dt.dayofyear.values)
doy_dict = {day: tmax_np[time.dt.dayofyear == day, ...] for day in doy}

CPU times: user 1.45 s, sys: 1.21 s, total: 2.65 s
Wall time: 2.35 s


# Various alternative calculation notes
The following cells show some attempts to speed up the calculation. These are basically not effective, but possibly because of the same fundamental problem with xarray.

In [71]:
%%time
# print(tmax.shape)
tmp = tmax.stack(z=('lat','lon'))   # makes it (time,  z)
tmp_good = np.isfinite(tmp)         # tests for valid data (=1)
tmp_good_sum = tmp_good.sum(axis=0) # counts valid values in each space point (shape = z)

print(tmp_good_sum)
print(tmp_good_sum.shape)
print(tmp_good_sum.max().values)

calc_points = np.nonzero(tmp_good_sum.values)[0] # this is where we have data to work with, returns z indices // Caution: returns a tuple of indices for each dimension, so have to use '[0]'
print(calc_points)
print(f"There are {len(calc_points)} that need to be processed.")

<xarray.DataArray 'tmax' (z: 259200)>
dask.array<shape=(259200,), dtype=int64, chunksize=(259200,)>
Coordinates:
  * z        (z) MultiIndex
  - lat      (z) float64 89.75 89.75 89.75 89.75 ... 89.75 89.75 89.75 89.75
  - lon      (z) float64 0.25 0.75 1.25 1.75 2.25 ... 13.25 13.75 14.25 14.75
(259200,)
3651
[  9286   9287   9288 ... 214506 214507 215226]
There are 68906 that need to be processed.
CPU times: user 1.49 s, sys: 108 ms, total: 1.6 s
Wall time: 8.77 s


In [ ]:
%%time
result = np.full(tmp_good_sum.shape, np.nan)
# this reduces the problem to 1-dimension, which is embarassingly parallel in theory.
for zpt in calc_points:
    good_times = np.where(tmp_good[:,zpt]==1)[0] # indices returned
    w = tmp[good_times, zpt]  # this is the valid data at spatial point zpt
    result[zpt] = np.percentile(w, 25) # does the (fast) calculation at zpt


# s = tmp.shape
# print(f"N = {s[1]}")
# result = []
# for j in np.arange(s[1]):
#     if good_vals_sum[] == 0:
#         result.append(np.nan)
#     else:
#         result.append(np.percentile(tmp[good_vals,j], 25))
# result = np.array(result)
result = xr.DataArray(result, coords=tmp_good_sum.coords, dims=tmp_good_sum.dims)
print(result)

In [62]:
result.max()

<xarray.DataArray ()>
array(nan)

In [11]:
%%time
tmax.load()

CPU times: user 13.2 s, sys: 25 s, total: 38.3 s
Wall time: 44.7 s


<xarray.DataArray 'tmax' (time: 14736, lat: 360, lon: 720)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 ... 358.25 358.75 359.25 359.75
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-05-06
Attributes:
    level_desc:    Surface
    statistic

In [9]:
%%time
grp = tmax.groupby('time.dayofyear')

CPU times: user 23.2 ms, sys: 947 µs, total: 24.1 ms
Wall time: 20.6 ms


In [14]:
%%time
# get the 90th percentile
tmax_90_by_day = grp.apply(xr.DataArray.quantile, args=(.9,), **{'dim':'time', 'interpolation':'linear', 'keep_attrs':True})

/project/amp/brianpm/miniconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1354: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


CPU times: user 1h 1min 23s, sys: 7min 43s, total: 1h 9min 7s
Wall time: 1h 21s


In [16]:
tmax_90_by_day.name = "tmax90pct"

In [18]:
# ONLY SAVE IF THIS IS FULL DATA SET -- NOT TEST
# tmax_90_by_day.to_netcdf("/project/amp/brianpm/TemperatureExtremes/Derived/CPC_tmax_90pct_dayofyear.nc")

## thoughts on this approach

I like this Xarray approach because it works in one line. The obvious downside is that it is unreasonably slow.

In our next step, we probably want to gather days on either side of each day to increase sample size. In Xarray, we can definitely do that by using a rolling operation, but that would need to be separate from the groupby operation, I think. 

Probably the easiest option is to deal with the time sampling in a loop. Then there's a question of speed. Is it worth breaking the quantile calculation into a loop and spreading it out using multiprocessing, or just wait it out without having to deal with setting up a pool.

In [38]:
tmax_pt = tmax[:,100,100]
tmax_pt

<xarray.DataArray 'tmax' (time: 14736)>
array([14.950293, 16.44022 , 19.941858, ..., 20.553688, 20.05624 , 22.033697],
      dtype=float32)
Coordinates:
    lat      float32 39.75
    lon      float32 50.25
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-05-06
Attributes:
    level_desc:    Surface
    statistic:     Mean
    parent_stat:   Other
    long_name:     Daily Maximum Temperature
    cell_methods:  time: mean
    valid_range:   [-90.  50.]
    avg_period:    0000-00-01 00:00:00
    dataset:       CPC Global Temperature
    comments:      GTS data and is gridded using the Shepard Algorithm
    max_period:    6z to 6z
    units:         degC
    var_desc:      Maximum Temperature

In [44]:
g = dict(tmax_pt.groupby("time.dayofyear"))  # this is kind of a kludge, but gives keys 1-366, with values that are dataArrays

In [50]:
tmax_by_day = dict(tmax.groupby("time.dayofyear"))  # this is kind of a kludge, but gives keys 1-366, with values that are dataArrays

In [54]:
tmax_90th_by_day = {k:np.nanquantile(tmax_by_day[k], 0.9, axis=0) for k in tmax_by_day}

In [10]:
%%time
# this is probably the same as above, but now we'll get a better data structure out of it.
# Slow when applied to the whole dataset.
tmax_90_by_day_2 = tmax.groupby('time.dayofyear').apply(xr.DataArray.quantile, args=(.9,), **{'dim':'time', 'interpolation':'linear', 'keep_attrs':True})

TypeError: quantile does not work for arrays stored as dask arrays. Load the data via .compute() or .load() prior to calling this method.

# More robust sampling

Now that we have a reasonably fast approach (by just using numpy arrays instead of DataArray), we want to next try to improve sampling by using a window around each day of the year. 
how to put a window around each day to increase the sample size and smooth the quantile time series



In [89]:
# # Use modulo operator (i.e., remainder) to deal with circular index.
# # https://stackoverflow.com/questions/8951020/pythonic-circular-list
def get_window_indices(thing, current, look_back, look_ahead):
    """Given an iterable, thing, return the values of thing in circular slice from look_back to look_ahead."""
    N = len(thing)-1
    window_inds = np.arange(-1*(look_back), look_ahead+1)
    result = []
    for w in window_inds:
        result.append(thing[(current + w) % N])
    return np.array(result)  # these are the values of thing

In [88]:
%%time
# Now we use our alternative to grouping to get good sampling.
# If we have around 50 years of data, but we want to calculate the 90th percentile, we have a couple of options:
# (1) Just do what we can and try to calculate confidence intervals for the quantiles we produce,
# (2) Increase sample size by using surrounding days (as in Perkins&Alexander),
# (3) DO BOTH!
# The issue is simply that we want a reasonably confident estimate of the quantile.

# Let's start by doing Option2 because it is a relatively simple extension of our code.
# If we have 50 years and we want a better estimate of quantiles, taking a 3-day window increases N to 150
# a 5 day window gets us to 250, which seems better.
# To loop across 31 Dec/1 Jan, we need to explicitly deal with it

tday = time.dt.dayofyear.values # day of year for every time.
doy = set(tday)  # kind of silly, but will deal with 365 or 366 day year
doy_list = list(doy)
doy_dict = dict()
Ndays = len(doy_list)-1
for i, day in enumerate(doy_list):
    use_days = get_window_indices(doy_list, i, 2, 2)
    use_inds = []
    for j in use_days:
        use_inds.append(np.nonzero(tday == j))
    use_inds = np.concatenate(use_inds).ravel()
    doy_dict[day] = tmax_np[use_inds, ...]
    

CPU times: user 3.71 s, sys: 4.03 s, total: 7.73 s
Wall time: 6.8 s


In [96]:
def get_our_quants(data):
        return np.nanquantile(data, [.01, .05, .1, .25, .5, .75, .9, .95, .99], axis=0, overwrite_input=False, interpolation='linear', keepdims=False)

In [93]:
import psutil
# logical=True counts threads, but we are interested in cores
psutil.cpu_count(logical=False)


In [91]:
# Now go through each day and calculate the quantiles:
doy_quants = dict()
for day in doy_dict:
    print(f"The day is {day}")
    doy_quants[day] = np.nanquantile(doy_dict[day], [.01, .05, .1, .25, .5, .75, .9, .95, .99], axis=0, 
                                     overwrite_input=False, interpolation='linear', keepdims=False)




The day is 1
The day is 2
The day is 3
The day is 4
The day is 5
The day is 6
The day is 7


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [111]:
%%time
# https://hpc-carpentry.github.io/hpc-python/06-parallel/
# https://github.com/uqfoundation/multiprocess
from multiprocess import Pool
pool = Pool(24)
doy_quants = pool.map(lambda x: get_our_quants(doy_dict[x]), doy_dict)
pool.close()

CPU times: user 22.3 s, sys: 14.1 s, total: 36.5 s
Wall time: 2min 46s


In [107]:
doy_quants

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [80]:
np.append(None, [1, 2])

array([None, 1, 2], dtype=object)